In [1]:
from __future__ import print_function

import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
tf.set_random_seed(777)  # reproducibility

In [2]:
sentences = ["그는 이날 기념식에서 아버지에게 바치는 ‘슬픈 생일’이라는 제목의 추모사를 낭독했다.",
            "김씨가 아버지에게 보내는 편지를 읽고 무대 뒤로 퇴장하려 하자 문 대통령은 자리에서 일어나 김씨를 따라가 안아주며 위로했다."]

char_set = list(set(' '.join(sentences)))
char_dic = {w: i for i, w in enumerate(char_set)}
print(char_dic)
# sentence = ("그는 이날 기념식에서 아버지에게 바치는 ‘슬픈 생일’이라는 제목의 추모사를 낭독했다.")
# char_set = list(set(sentence))
# char_dic = {w: i for i, w in enumerate(char_set)}
# print(char_dic)

{'며': 1, '장': 53, '바': 2, '대': 3, '의': 4, '일': 5, '치': 6, '주': 7, '날': 8, '생': 9, '를': 11, '무': 12, '령': 13, '김': 16, '슬': 15, '독': 17, '라': 18, '려': 20, '목': 21, '기': 22, '식': 23, '따': 24, '하': 0, '서': 25, '내': 26, '뒤': 27, '통': 28, '‘': 30, ' ': 61, '아': 31, '버': 14, '은': 33, '로': 32, '사': 35, '지': 29, '어': 40, '보': 39, '나': 34, '문': 41, '퇴': 42, '낭': 43, '고': 44, '이': 45, '가': 46, '리': 47, '게': 48, '씨': 49, '편': 56, '.': 19, '읽': 51, '제': 52, '자': 55, '했': 54, '그': 36, '위': 50, '에': 10, '는': 57, '념': 58, '’': 59, '추': 37, '안': 60, '다': 38, '픈': 62, '모': 63}


In [3]:
chosung_sentences = ["ㄱㄴ ㅇㄴ ㄱㄴㅅㅇㅅ ㅇㅂㅈㅇㄱ ㅂㅊㄴ ‘ㅅㅍ ㅅㅇ’ㅇㄹㄴ ㅈㅁㅇ ㅊㅁㅅㄹ ㄴㄷㅎㄷ.",
                    "ㄱㅆㄱ ㅇㅂㅈㅇㄱ ㅂㄴㄴ ㅍㅈㄹ ㅇㄱ ㅁㄷ ㄷㄹ ㅌㅈㅎㄹ ㅎㅈ ㅁ ㄷㅌㄹㅇ ㅈㄹㅇㅅ ㅇㅇㄴ ㄱㅆㄹ ㄸㄹㄱ ㅇㅇㅈㅁ ㅇㄹㅎㄷ."]
chosung_set = list(set(' '.join(chosung_sentences)))
chosung_dic = {w: i for i, w in enumerate(chosung_set)}
print(chosung_dic)
# chosung_sentence = ("ㄱㄴ ㅇㄴ ㄱㄴㅅㅇㅅ ㅇㅂㅈㅇㄱ ㅂㅊㄴ ‘ㅅㅍ ㅅㅇ’ㅇㄹㄴ ㅈㅁㅇ ㅊㅁㅅㄹ ㄴㄷㅎㄷ.")
# chosung_set = list(set(chosung_sentence))
# chosung_dic = {w: i for i, w in enumerate(chosung_set)}
# print(chosung_dic)

{'ㄱ': 12, 'ㄷ': 0, 'ㅇ': 16, '.': 3, 'ㅎ': 4, 'ㄸ': 5, 'ㅍ': 6, 'ㅈ': 18, 'ㅂ': 17, 'ㅆ': 1, 'ㅌ': 9, '’': 7, '‘': 10, 'ㅅ': 11, 'ㄹ': 8, 'ㅊ': 14, 'ㄴ': 2, 'ㅁ': 15, ' ': 13}


In [4]:
data_dim = len(char_set)
hidden_size = len(char_set)
num_classes = len(char_set)
sequence_length = 10  # Any arbitrary number
learning_rate = 0.05

In [5]:
dataX = []
dataY = []
for j in range(len(sentences)):
    sentence = sentences[j]
    chosung_sentence = chosung_sentences[j]
    for i in range(0, len(sentence) - sequence_length):
        x_str = sentence[i:i + sequence_length]
        y_str = chosung_sentence[i:i + sequence_length]
        print(i, x_str, '->', y_str)

        x = [char_dic[c] for c in x_str]  # x str to index
        y = [chosung_dic[c] for c in y_str]  # y str to index

        dataX.append(x)
        dataY.append(y)

0 그는 이날 기념식에 -> ㄱㄴ ㅇㄴ ㄱㄴㅅㅇ
1 는 이날 기념식에서 -> ㄴ ㅇㄴ ㄱㄴㅅㅇㅅ
2  이날 기념식에서  ->  ㅇㄴ ㄱㄴㅅㅇㅅ 
3 이날 기념식에서 아 -> ㅇㄴ ㄱㄴㅅㅇㅅ ㅇ
4 날 기념식에서 아버 -> ㄴ ㄱㄴㅅㅇㅅ ㅇㅂ
5  기념식에서 아버지 ->  ㄱㄴㅅㅇㅅ ㅇㅂㅈ
6 기념식에서 아버지에 -> ㄱㄴㅅㅇㅅ ㅇㅂㅈㅇ
7 념식에서 아버지에게 -> ㄴㅅㅇㅅ ㅇㅂㅈㅇㄱ
8 식에서 아버지에게  -> ㅅㅇㅅ ㅇㅂㅈㅇㄱ 
9 에서 아버지에게 바 -> ㅇㅅ ㅇㅂㅈㅇㄱ ㅂ
10 서 아버지에게 바치 -> ㅅ ㅇㅂㅈㅇㄱ ㅂㅊ
11  아버지에게 바치는 ->  ㅇㅂㅈㅇㄱ ㅂㅊㄴ
12 아버지에게 바치는  -> ㅇㅂㅈㅇㄱ ㅂㅊㄴ 
13 버지에게 바치는 ‘ -> ㅂㅈㅇㄱ ㅂㅊㄴ ‘
14 지에게 바치는 ‘슬 -> ㅈㅇㄱ ㅂㅊㄴ ‘ㅅ
15 에게 바치는 ‘슬픈 -> ㅇㄱ ㅂㅊㄴ ‘ㅅㅍ
16 게 바치는 ‘슬픈  -> ㄱ ㅂㅊㄴ ‘ㅅㅍ 
17  바치는 ‘슬픈 생 ->  ㅂㅊㄴ ‘ㅅㅍ ㅅ
18 바치는 ‘슬픈 생일 -> ㅂㅊㄴ ‘ㅅㅍ ㅅㅇ
19 치는 ‘슬픈 생일’ -> ㅊㄴ ‘ㅅㅍ ㅅㅇ’
20 는 ‘슬픈 생일’이 -> ㄴ ‘ㅅㅍ ㅅㅇ’ㅇ
21  ‘슬픈 생일’이라 ->  ‘ㅅㅍ ㅅㅇ’ㅇㄹ
22 ‘슬픈 생일’이라는 -> ‘ㅅㅍ ㅅㅇ’ㅇㄹㄴ
23 슬픈 생일’이라는  -> ㅅㅍ ㅅㅇ’ㅇㄹㄴ 
24 픈 생일’이라는 제 -> ㅍ ㅅㅇ’ㅇㄹㄴ ㅈ
25  생일’이라는 제목 ->  ㅅㅇ’ㅇㄹㄴ ㅈㅁ
26 생일’이라는 제목의 -> ㅅㅇ’ㅇㄹㄴ ㅈㅁㅇ
27 일’이라는 제목의  -> ㅇ’ㅇㄹㄴ ㅈㅁㅇ 
28 ’이라는 제목의 추 -> ’ㅇㄹㄴ ㅈㅁㅇ ㅊ
29 이라는 제목의 추모 -> ㅇㄹㄴ ㅈㅁㅇ ㅊㅁ
30 라는 제목의 추모사 -> ㄹㄴ ㅈㅁㅇ ㅊㅁㅅ
31 는 제목의 추모사를 -> ㄴ ㅈㅁㅇ ㅊㅁㅅㄹ
32  제목의 추모사를  ->  ㅈㅁㅇ ㅊㅁㅅㄹ 
33 제목의 추모사를 낭 -> ㅈㅁㅇ ㅊㅁㅅㄹ ㄴ
34 목의 추모사를 낭독 -> ㅁㅇ ㅊㅁㅅㄹ ㄴㄷ
35 의 추모사를 낭독했 -> ㅇ ㅊㅁㅅㄹ ㄴㄷㅎ
36

In [6]:
test_sentence = "아버지에게 바치는 ‘슬픈 생일’이라는 제목의 추모사를 그는 이날 기념식에서 낭독했다."
test_chosung_sentence = "ㅇㅂㅈㅇㄱ ㅂㅊㄴ ‘ㅅㅍ ㅅㅇ’ㅇㄹㄴ ㅈㅁㅇ ㅊㅁㅅㄹ ㄱㄴ ㅇㄴ ㄱㄴㅅㅇㅅ ㄴㄷㅎㄷ."

test_dataX = []
test_dataY = []
for i in range(0, len(test_sentence) - sequence_length):
    x_str = test_sentence[i:i + sequence_length]
    y_str = test_chosung_sentence[i:i + sequence_length]
    print(i, x_str, '->', y_str)

    x = [char_dic[c] for c in x_str]  # x str to index
    y = [chosung_dic[c] for c in y_str]  # y str to index

    test_dataX.append(x)
    test_dataY.append(y)

0 아버지에게 바치는  -> ㅇㅂㅈㅇㄱ ㅂㅊㄴ 
1 버지에게 바치는 ‘ -> ㅂㅈㅇㄱ ㅂㅊㄴ ‘
2 지에게 바치는 ‘슬 -> ㅈㅇㄱ ㅂㅊㄴ ‘ㅅ
3 에게 바치는 ‘슬픈 -> ㅇㄱ ㅂㅊㄴ ‘ㅅㅍ
4 게 바치는 ‘슬픈  -> ㄱ ㅂㅊㄴ ‘ㅅㅍ 
5  바치는 ‘슬픈 생 ->  ㅂㅊㄴ ‘ㅅㅍ ㅅ
6 바치는 ‘슬픈 생일 -> ㅂㅊㄴ ‘ㅅㅍ ㅅㅇ
7 치는 ‘슬픈 생일’ -> ㅊㄴ ‘ㅅㅍ ㅅㅇ’
8 는 ‘슬픈 생일’이 -> ㄴ ‘ㅅㅍ ㅅㅇ’ㅇ
9  ‘슬픈 생일’이라 ->  ‘ㅅㅍ ㅅㅇ’ㅇㄹ
10 ‘슬픈 생일’이라는 -> ‘ㅅㅍ ㅅㅇ’ㅇㄹㄴ
11 슬픈 생일’이라는  -> ㅅㅍ ㅅㅇ’ㅇㄹㄴ 
12 픈 생일’이라는 제 -> ㅍ ㅅㅇ’ㅇㄹㄴ ㅈ
13  생일’이라는 제목 ->  ㅅㅇ’ㅇㄹㄴ ㅈㅁ
14 생일’이라는 제목의 -> ㅅㅇ’ㅇㄹㄴ ㅈㅁㅇ
15 일’이라는 제목의  -> ㅇ’ㅇㄹㄴ ㅈㅁㅇ 
16 ’이라는 제목의 추 -> ’ㅇㄹㄴ ㅈㅁㅇ ㅊ
17 이라는 제목의 추모 -> ㅇㄹㄴ ㅈㅁㅇ ㅊㅁ
18 라는 제목의 추모사 -> ㄹㄴ ㅈㅁㅇ ㅊㅁㅅ
19 는 제목의 추모사를 -> ㄴ ㅈㅁㅇ ㅊㅁㅅㄹ
20  제목의 추모사를  ->  ㅈㅁㅇ ㅊㅁㅅㄹ 
21 제목의 추모사를 그 -> ㅈㅁㅇ ㅊㅁㅅㄹ ㄱ
22 목의 추모사를 그는 -> ㅁㅇ ㅊㅁㅅㄹ ㄱㄴ
23 의 추모사를 그는  -> ㅇ ㅊㅁㅅㄹ ㄱㄴ 
24  추모사를 그는 이 ->  ㅊㅁㅅㄹ ㄱㄴ ㅇ
25 추모사를 그는 이날 -> ㅊㅁㅅㄹ ㄱㄴ ㅇㄴ
26 모사를 그는 이날  -> ㅁㅅㄹ ㄱㄴ ㅇㄴ 
27 사를 그는 이날 기 -> ㅅㄹ ㄱㄴ ㅇㄴ ㄱ
28 를 그는 이날 기념 -> ㄹ ㄱㄴ ㅇㄴ ㄱㄴ
29  그는 이날 기념식 ->  ㄱㄴ ㅇㄴ ㄱㄴㅅ
30 그는 이날 기념식에 -> ㄱㄴ ㅇㄴ ㄱㄴㅅㅇ
31 는 이날 기념식에서 -> ㄴ ㅇㄴ ㄱㄴㅅㅇㅅ
32  이날 기념식에서  ->  ㅇㄴ ㄱㄴㅅㅇㅅ 
33 이날 기념식에서 낭 -> ㅇㄴ ㄱㄴㅅㅇㅅ ㄴ
34 날 기념식에서 낭독 -> ㄴ ㄱㄴㅅㅇㅅ ㄴㄷ
35  기념식에서 낭독했 ->  ㄱㄴㅅㅇㅅ ㄴㄷㅎ
36

In [7]:
batch_size = len(dataX)

X = tf.placeholder(tf.int32, [None, sequence_length])
Y = tf.placeholder(tf.int32, [None, sequence_length])

# One-hot encoding
X_one_hot = tf.one_hot(X, num_classes)
print(X_one_hot)  # check out the shape

Tensor("one_hot:0", shape=(?, 10, 64), dtype=float32)


In [8]:
# Make a lstm cell with hidden_size (each unit output vector size)
cell = rnn.BasicLSTMCell(hidden_size, state_is_tuple=True)
cell = rnn.MultiRNNCell([cell] * 2, state_is_tuple=True)

# outputs: unfolding size x hidden size, state = hidden size
outputs, _states = tf.nn.dynamic_rnn(cell, X_one_hot, dtype=tf.float32)

# FC layer
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
outputs = tf.contrib.layers.fully_connected(X_for_fc, num_classes, activation_fn=None)

# reshape out for sequence_loss
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])

# All weights are 1 (equal weights)
weights = tf.ones([batch_size, sequence_length])

sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
mean_loss = tf.reduce_mean(sequence_loss)
train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(mean_loss)

In [9]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(5000):
    _, l, results = sess.run(
        [train_op, mean_loss, outputs], feed_dict={X: dataY, Y: dataX})
    if i % 100 == 0:
        print('step:',i,' loss:',l)
        results = sess.run(outputs, feed_dict={X: dataY})
        for j, result in enumerate(results):
            index = np.argmax(result, axis=1)
            if j is 0:  # print all for the first result to make a sentence
                print(''.join([char_set[t] for t in index]), end='')
            else:
                print(char_set[index[-1]], end='')
        print("\n\n")

step: 0  loss: 4.1599
                                                                                                         


step: 100  loss: 0.144541
가념 이날 기념식에서 아버지에게 바치는 ‘슬픈 생일’이라는 제목의 추모사를 낭독했다 보내는 편지를 읽고 무대 뒤로 퇴장하려 하자 문 대통령은 자리에서 일어나 김씨를 따라가 안아주며 위로했다


step: 200  loss: 0.143599
게념 이날 기념식에서 아버지에게 바치는 ‘슬픈 생일’이라는 제목의 추모사를 낭독했다 보내는 편지를 읽고 무대 뒤로 퇴장하려 하자 문 대통령은 자리에서 일어나 김씨를 따라가 안아주며 위로했다


step: 300  loss: 0.143379
가는 이날 기념식에서 아버지에게 바치는 ‘슬픈 생일’이라는 제목의 추모사를 낭독했다 보내는 편지를 읽고 무대 뒤로 퇴장하려 하자 문 대통령은 자리에서 일어나 김씨를 따라가 안아주며 위로했다


step: 400  loss: 0.143264
게념 이날 기념식에서 아버지에게 바치는 ‘슬픈 생일’이라는 제목의 추모사를 낭독했다 보내는 편지를 읽고 무대 뒤로 퇴장하려 하자 문 대통령은 자리에서 일어나 김씨를 따라가 안아주며 위로했다


step: 500  loss: 0.143208
게념 이날 기념식에서 아버지에게 바치는 ‘슬픈 생일’이라는 제목의 추모사를 낭독했다 보내는 편지를 읽고 무대 뒤로 퇴장하려 하자 문 대통령은 자리에서 일어나 김씨를 따라가 안아주며 위로했다


step: 600  loss: 0.143259
게념 이날 기념식에서 아버지에게 바치는 ‘슬픈 생일’이라는 제목의 추모사를 낭독했다 보내는 편지를 읽고 무대 뒤로 퇴장하려 하자 문 대통령은 자리에서 일어나 김씨를 따라가 안아주며 위로했다


step: 700  loss: 0.143134
게는 이날 기념식에서 아버지에게 바치는 ‘슬픈 생일’이라는 제목의 추모사